In [24]:
import re

from math import sqrt


SPEED_OF_SOUND = 343

NOTE_RE = re.compile("^([A-G][b#]?)(\d*)$")
SEMITONES_FROM_A = {
    "C": -9,
    "C#": -8,
    "Db": -8,
    "D": -7,
    "D#": -6,
    "Eb": -6,
    "E": -5,
    "F": -4,
    "F#": -3,
    "Gb": -3,
    "G": -2,
    "G#": -1,
    "Ab": -1,
    "A": 0,
    "A#": 1,
    "Bb": 1,
    "B": 2,
}

### Reverb

Realistic predelay times

In [25]:
def close_predelay_to_room_width(predelay_time_ms):
    return predelay_time_ms / 1000 * SPEED_OF_SOUND

In [26]:
room_width = close_predelay_to_room_width(50)
int(room_width)

17

In [27]:
def distance_to_predelay(room_width, distance_to_listener):
    """Given a room of `room_width` meters and the sound source is `distance_to_listener'
    meters away. What time (in ms) will the early reflection arrive after the direct source"""
    early_reflection = sqrt(distance_to_listener**2 + room_width**2)
    path_difference = early_reflection - distance_to_listener
    return path_difference / SPEED_OF_SOUND * 1000
    

In [28]:
# If the room 20m wide and the listener and sound source are central
# what pre-delay would mimic a source next to the listener
int(distance_to_predelay(room_width=20, distance_to_listener=0))

58

In [29]:
# What about if the source was 10m back...
int(distance_to_predelay(20, distance_to_listener=10))

36

In [30]:
def predelay_to_distance(width, predelay_time_ms):
    """Given a predelay time in ms - what distance to the source is it simulating?"""
    predelay_time = predelay_time_ms / 1000
    res = width**2 - SPEED_OF_SOUND**2 * predelay_time**2
    res /= 2 * SPEED_OF_SOUND * predelay_time
    return res

In [31]:
int(predelay_to_distance(20, 36))

10

In [32]:
# With a very short pre-delay such that the source and reflection arrive nearly the same time.
int(predelay_to_distance(20, 2))

291

### Compression

Tools for working out ratios when working with compression

In [33]:
def get_output_level(threshold, ratio, input_level):
    """What output level would I get with the threshold and ratio?"""
    if input_level <= threshold:
        return input_level
    
    return threshold  + (input_level - threshold) / ratio

In [34]:
# If the input level is below the threshold it won't change...
output_level(threshold=-6, ratio=2, input_level=-7)

-7

In [35]:
# Above the threshold the signal is compressed
output_level(threshold=-6, ratio=2, input_level=-2)

-4.0

In [36]:
def get_threshold(ratio, input_level, output_level):
    """What threshold should I set for a desired output level?"""
    assert input_level > output_level
    return (ratio * output_level - input_level) / (ratio - 1)

In [37]:
# From the above example what threshold would I set to get a 2dB reduction with a 2:1 ratio
get_threshold(ratio=2, input_level=-2, output_level=-4)

-6.0

### Frequency

In [38]:
def equal_temp_note_freq(note, a4_freq=440):
    """What's the frequency of a note (format: LETTEROCTAVE eg C#4)"""
    match = NOTE_RE.match(note)
    if not match:
        raise ValueError(f'Invalid Note format: {note}, should be of the form LETTEROCTAVE (Gb3)')
    
    note, num = match.groups()
    num = int(num)
    a_freq = a4_freq * 2**(num - 4)
    return a_freq * 2**(SEMITONES_FROM_A[note]/12)

In [39]:
equal_temp_note_freq("A4")

440.0

In [40]:
equal_temp_note_freq("C2")

65.40639132514966

In [42]:
def beat_freq_between_notes(note_1, note_2):
    """Given two notes - what is the frequency (in Hz) of the resulting beats
    as they go in and out of phase"""
    return abs(equal_temp_note_freq(note_1) - equal_temp_note_freq(note_2))

In [43]:
#What would the frequency of the beats between A4 and A#4
beat_freq_between_notes('A4', 'A#4')

26.163761518089927